# Fine stable diffusion XL

In [ ]:
import ads
import oci
import os
from oci.object_storage import ObjectStorageClient
from datetime import datetime, timedelta

ads.set_auth(auth='resource_principal')
rps = oci.auth.signers.get_resource_principals_signer()

# Object storage client
object_storage_client = ObjectStorageClient(config={}, signer=rps)
OBJECT_STORAGE_NAMESPACE = object_storage_client.get_namespace().data

os.environ['namespace'] = OBJECT_STORAGE_NAMESPACE
LOG_GROUP_ID = os.environ['loggroup_ocid']
LOG_ID = os.environ['log_ocid']
BUCKET_NAME = os.environ['bucket_name']

INPUT_FOLDER = "oci://{bucket}@{namespace}/sdxl/input/".format(bucket=BUCKET_NAME, namespace=OBJECT_STORAGE_NAMESPACE)


In [ ]:
import ocifs
fs = ocifs.OCIFileSystem()
fs.makedir('sdxl/input', create_parents=True)
fs.makedir('sdxl/output', create_parents=True)

In [ ]:

!git clone https://github.com/bmaltais/kohya_ss.git /home/datascience/job_artifact/kohya_ss
!odsc conda install --s pytorch20_p39_gpu_v1
!conda activate pytorch20_p39_gpu_v1
!cd /home/datascience/job_artifact/kohya_ss && bash setup.sh
!pip uninstall opencv-python && pip install opencv-python-headless # 
!odsc conda init -b $bucket_name -n $namespace -a resource_principal
!odsc conda publish -s pytorch20_p39_gpu_v1 --force

In [ ]:
from ads.jobs import Job, DataScienceJob, PythonRuntime

job = (
    Job(name="sdxl-train-job")
    .with_infrastructure(
        DataScienceJob()
        .with_log_group_id(LOG_GROUP_ID)
        .with_log_id(LOG_ID)
        .with_shape_name("VM.GPU2.1")
    )
    .with_runtime(
        PythonRuntime()
        .with_custom_conda("oci://{bucket}@{namespace}/conda_environments/gpu/PyTorch 2.0 for GPU on Python 3.9/1.0/pytorch20_p39_gpu_v1".format(bucket=BUCKET_NAME, namespace=OBJECT_STORAGE_NAMESPACE))
        .with_source("/home/datascience/job_artifact/")
        .with_entrypoint("stable_main.py")
        .with_working_dir("job_artifact")
        .with_environment_variable(full_input_folder=INPUT_FOLDER)
        .with_output("./output", "oci://{bucket}@{namespace}/sdxl/output/".format(bucket=BUCKET_NAME, namespace=OBJECT_STORAGE_NAMESPACE))
    )
)

job.create()

In [ ]:
job_run_env = job.run(
    name="Job Run - Passing dynamic values",
    env_var={'full_input_folder': INPUT_FOLDER}
)

job_run_watch = job_run_env.watch()